# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,25.25,47,7,5.90,MG,1690707600
1,1,bac lieu,9.2850,105.7244,31.32,61,98,5.68,VN,1690783200
2,2,ta`u,-14.2336,-169.5144,27.48,72,88,4.44,AS,1690783200
3,3,bloemfontein,-29.1211,26.2140,24.07,20,33,10.06,ZA,1690545600
4,4,batticaloa,7.7102,81.6924,33.04,47,98,0.71,LK,1690610400


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
# Configure the map plot_3
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map plot
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Humidity"] <= 50)]

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,25.25,47,7,5.90,MG,1690707600
3,3,bloemfontein,-29.1211,26.2140,24.07,20,33,10.06,ZA,1690545600
4,4,batticaloa,7.7102,81.6924,33.04,47,98,0.71,LK,1690610400
7,7,evans,33.5337,-82.1307,39.90,26,12,3.38,US,1690750800
10,10,vilyuysk,63.7553,121.6247,26.67,40,51,2.05,RU,1690524000


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = ideal_weather_df.copy()
#columns = ["City", "Country", "Lat", "Lng","Humidity"]
hotels_df = hotels_df[
    [
        "City",
        "Country",
        "Lat",
        "Lng",
        "Humidity"
    ]
]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df["Hotel Name"] = " "

# Display sample data
hotels_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tolanaro,MG,-25.0319,46.9987,47,
3,bloemfontein,ZA,-29.1211,26.2140,20,
4,batticaloa,LK,7.7102,81.6924,47,
7,evans,US,33.5337,-82.1307,26,
10,vilyuysk,RU,63.7553,121.6247,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
tolanaro - nearest hotel: Tôlanaro Airport
bloemfontein - nearest hotel: Bram Fischer International Airport
batticaloa - nearest hotel: Batticaloa Airport
evans - nearest hotel: No hotel found
vilyuysk - nearest hotel: Аэропорт Вилюйск
murun-kuren - nearest hotel: No hotel found
moro - nearest hotel: No hotel found
nyrob - nearest hotel: No hotel found
khorixas - nearest hotel: No hotel found
queenstown - nearest hotel: Queenstown Airport
yellowknife - nearest hotel: Yellowknife Airport
san patricio - nearest hotel: No hotel found
nagoya - nearest hotel: 県営名古屋空港
constantia - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
tekeli - nearest hotel: No hotel found
fruita - nearest hotel: No hotel found
bethel - nearest hotel: Danbury Municipal Airport
college - nearest hotel: Fairbanks International Airport
estevan - nearest hotel: Estevan Regional Airport
nguigmi - nearest hotel: No hotel found
bonnyville - nearest hotel: Bonnyville Airport
ne

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Configure the map plot
# Configure the map plot_4
map_plot_4 = hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map plot
map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)